# 한글-Claude-v2 Model: Text summarization with small files with Anthropic Claude

---

### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

이 예에서는 소량의 데이터(문자열 데이터)를 Amazon Bedrock API(Anthropic Claude 모델 사용)로 직접 수집하고 각 텍스트를 요약하라는 지침을 제공합니다.

### 건축학

![](./images/41-text-simple-1.png)

이 아키텍처에서:

1. 작은 텍스트(또는 작은 파일)가 로드됨
1. 입력 데이터를 처리하는 기본 모델
1. 모델은 수집된 텍스트의 요약과 함께 응답을 반환합니다.

### 사용 사례

이 접근 방식은 통화 기록, 회의 기록, 서적, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

### 도전

이 접근 방식은 입력 텍스트 또는 파일이 모델 컨텍스트 길이에 맞을 때 사용할 수 있습니다. 노트북 `02.long-text-summarization-titan.ipynb`에서는 사용자가 토큰 한도를 초과하는 대용량 문서를 가지고 있을 때 문제를 해결하기 위한 접근 방식을 탐색합니다.

## 1. Bedrock Client 생성

In [1]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## 2. boto3로 짧은 텍스트 요약하기
 
Amazon Bedrock에 대한 API 요청의 세부 사항을 알아보기 위해 이 노트북에서는 API 요청을 생성하고 Boto3 작업을 래핑하여 더 간단한 API를 제공하는 langchain에 의존하지 않고 Boto3를 통해 요청을 보내는 방법을 소개합니다.

### Boto3에서 InvokeModel의 요청 구문


기초 모델에 요청을 보내기 위해 `InvokeModel` API를 사용합니다. 다음은 Anthropic Claude에게 텍스트를 보내기 위한 API 요청의 예입니다. `textGenerationConfig`의 추론 매개변수는 사용하려는 모델에 따라 다릅니다. Anthropic Claude의 추론 매개변수는 다음과 같습니다.

- **temperature**는 생성의 임의성 정도를 조정합니다. 낮은 temperature는 무작위 생성이 적다는 것을 의미합니다.
- **top_p** 1 미만은 생성을 위해 top_p 이상까지 합산되는 확률을 가진 가장 가능성 있는 토큰의 가장 작은 집합만 유지합니다.
- **top_k**는 생성된 토큰의 반복성을 줄이기 위해 사용될 수 있습니다. 값이 높을수록 프롬프트 또는 이전 세대에 이미 나타난 횟수에 비례하여 이전에 존재하는 토큰에 더 강력한 패널티가 적용됩니다.
- **max_tokens_to_sample**은 생성할 최대 토큰 수입니다. 응답이 원하는 최대 길이까지 채워지는 것은 아닙니다.
- **stop_sequences**는 API가 추가 토큰 생성을 중지하는 시퀀스입니다. 반환된 텍스트에는 중지 시퀀스가 ​​포함되지 않습니다.

```python
response = bedrock.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v1", 
                                accept=accept, 
                                contentType=contentType)

```

### 요약할 텍스트가 있는 쓰기 프롬프트

이 노트북에서는 토큰이 기초 모델의 최대 토큰보다 작은 짧은 텍스트를 사용할 수 있습니다. 짧은 텍스트의 예로 [AWS 블로그 게시물](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) Amazon Bedrock 발표에 대해.

프롬프트는 '다음 텍스트에 대한 요약을 제공하십시오.'라는 지시로 시작됩니다.

In [2]:
prompt = """
다음 텍스트의 요약을 제공하십시오.

AWS는 고객의 모든 피드백을 받아 오늘 Amazon Bedrock을 발표하게 되어 기쁩니다.
API를 통해 AI21 Labs, Anthropic, Stability AI 및 Amazon의 FM에 액세스할 수 있는 새로운 서비스입니다. \
Bedrock은 고객이 FM을 사용하여 생성 AI 기반 애플리케이션을 구축하고 확장하는 가장 쉬운 방법입니다.
모든 빌더의 액세스 민주화. Bedrock은 다양한 강력한 FM에 액세스할 수 있는 기능을 제공합니다 \
두 개의 새로운 LLM으로 구성된 Amazons Titan FM을 포함한 텍스트 및 이미지용 \
확장 가능하고 안정적이며 안전한 AWS 관리형 서비스를 통해 Bedrock의 서버리스 경험을 통해 \
고객은 수행하려는 작업에 적합한 모델을 쉽게 찾고 개인적으로 신속하게 시작할 수 있습니다 \
자체 데이터로 FM을 사용자 지정하고 AWS \
인프라를 관리할 필요 없이 익숙한 도구 및 기능(통합 \
Amazon SageMaker ML 기능(예: 다양한 모델을 테스트하기 위한 실험 및 규모에 맞게 FM을 관리하기 위한 파이프라인).
"""

In [3]:
print("prompt: \n", prompt)

prompt: 
 
다음 텍스트의 요약을 제공하십시오.

AWS는 고객의 모든 피드백을 받아 오늘 Amazon Bedrock을 발표하게 되어 기쁩니다.
API를 통해 AI21 Labs, Anthropic, Stability AI 및 Amazon의 FM에 액세스할 수 있는 새로운 서비스입니다. Bedrock은 고객이 FM을 사용하여 생성 AI 기반 애플리케이션을 구축하고 확장하는 가장 쉬운 방법입니다.
모든 빌더의 액세스 민주화. Bedrock은 다양한 강력한 FM에 액세스할 수 있는 기능을 제공합니다 두 개의 새로운 LLM으로 구성된 Amazons Titan FM을 포함한 텍스트 및 이미지용 확장 가능하고 안정적이며 안전한 AWS 관리형 서비스를 통해 Bedrock의 서버리스 경험을 통해 고객은 수행하려는 작업에 적합한 모델을 쉽게 찾고 개인적으로 신속하게 시작할 수 있습니다 자체 데이터로 FM을 사용자 지정하고 AWS 인프라를 관리할 필요 없이 익숙한 도구 및 기능(통합 Amazon SageMaker ML 기능(예: 다양한 모델을 테스트하기 위한 실험 및 규모에 맞게 FM을 관리하기 위한 파이프라인).



## 3. 프롬프트 및 추론 매개변수로 요청 본문 만들기

'invoke_model'의 요청 구문에 따라 위의 프롬프트 및 추론 매개변수를 사용하여 요청 본문을 생성합니다.

In [4]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

## 4. Boto3를 통해 기초 모델 호출

여기에서 요청 파라미터 `modelId`, `accept` 및 `contentType`을 지정하여 Amazon Bedrock에 API 요청을 보냅니다. 프롬프트에 따라 Amazon Bedrock의 기초 모델이 텍스트를 요약합니다.

In [5]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

Bedrock은 고객이 책임감 있고 윤리적으로 AI를 구축하고 배포할 수 있도록 지원합니다.

요약:

- AWS는 Amazon Bedrock이라는 새로운 서비스를 출시했습니다.
- Bedrock을 통해 고객은 AI21 Labs, Anthropic, Stability AI 및 Amazon의 다양한 기능 강화 모델(FM)에 API를 통해 액세스할 수
있습니다.
- Bedrock은 고객이 FM을 사용하여 AI 기반 애플리케이션을 쉽게 구축하고 확장할 수 있도록 지원합니다.
- Bedrock은 서버리스 경험을 제공하며, 고객은 자신의 데이터로 FM을 사용자 지정하고 AWS 인프라 관리 없이 작업을 빠르게 시작할 수 있습니다.
- Bedrock은 고객이 책임감 있고 윤리적으로 AI를 구축하고 배포할 수 있도록 지원합니다.


## 5. Stream 으로 요약을 생성하기

위에서 Bedrock 서비스는 주어진 프롬프트에 대한 전체 요약을 단일 출력으로 생성합니다. 출력에 많은 양의 토큰이 포함된 경우 속도가 느려질 수 있습니다.

아래에서는 Bedrock을 사용하여 출력을 스트리밍하여 모델에서 생성될 때 사용자가 소비를 시작할 수 있는 옵션을 살펴봅니다. 이를 위해 Bedrock은 청크 형태로 출력을 스트리밍하는 'ResponseStream'을 제공하는 'invoke_model_with_response_stream' API를 지원합니다.

전체 출력을 생성하는 대신 Bedrock은 모델에서 더 작은 청크를 보냅니다. 이것은 소모품 방식으로도 표시될 수 있습니다.

In [6]:
from IPython.display import display_markdown,Markdown,clear_output

In [7]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

Bedrock은 고객이 AI를 더 빠르고 쉽게 활용할 수 있도록 지원하며, 모든 사람이 AI를 더 안전하고 책임감 있게 사용할 수 있도록 하는 데 전념하고 있습니다.

요약:

Amazon은 고객의 피드백을 바탕으로 Amazon Bedrock이라는 새로운 서비스를 발표했습니다. Bedrock은 API를 통해 AI21 Labs, Anthropic, Stability AI 및 Amazon의 Titan FM 등 다양한 강력한 FM에 액세스할 수 있는 관리형 서비스입니다. 이 서비스를 통해 고객은 쉽게 적합한 모델을 찾고 FM을 사용하여 AI 기반 애플리케이션을 빠르게 구축 및 확장할 수 있습니다. 또한 고객 데이터로 FM을 사용자 지정하고 SageMaker와 같은 도구를 활용할 수 있습니다. Amazon은 Bedrock을 통해 고객이 AI를 더욱 쉽고 빠르게 활용할 수 있도록 지원하면서도 안전하고 책임감 있는 방식으로 사용할 수 있도록 노력하고 있습니다.

## 6. 결론
이제 Amazon Bedrock API에 바닐라 노출을 제공하는 `boto3` SDK를 사용하여 실험했습니다. 이 API를 사용하여 전체 출력 및 스트리밍 출력 생성이라는 두 가지 방법으로 Amazon Bedrock에 대한 AWS 뉴스 요약을 생성하는 사용 사례를 확인했습니다.

### Next Action
- 프롬프트를 특정 사용 사례로 변경하고 다른 모델의 출력을 평가합니다.
- 서비스의 대기 시간과 응답성을 이해하기 위해 토큰 길이를 가지고 놀아보세요.
- 더 나은 결과를 얻기 위해 다양한 프롬프트 엔지니어링 원칙을 적용합니다.